# Le pone los datos del partido a todos los registros tratando partición a partición

In [1]:
### Probando con datos
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

conf = (SparkConf().set("spark.executor.memory", "6g")   #<--- Por executor. Como hay dos instancias, la mitad por executor.
                    .set("spark.executor.cores", "3")    #<--- Por cada executor. Como hay 4, 4*3=12 cores se usan.
                    .set("spark.executor.instances", "2")
                    .set("spark.eventLog.enabled", "true"))
spark = SparkSession.builder.master("spark://spark-master:7077").appName("ReadParquetYearMon16MM").config(conf=conf).getOrCreate()
spark

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/08 17:51:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import time
print ('Inicio: '+time.strftime("%c"))
inicio = time.perf_counter()

Inicio: Tue Nov  8 17:52:01 2022


In [3]:
%%time
from pyspark.sql.types import StructType
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType
from pyspark.sql.functions import sum, col, desc

esqPart = StructType() \
      .add("partido",StringType(),True) \
      .add("nombre",StringType(),True) \
      .add("lider",StringType(),True) \
      .add("fechaCreacion",StringType(),True)

dfPart = spark.read.csv("./datos/partidos.csv", sep=";", header = True, schema = esqPart)

dfPart.show()

esquema = StructType() \
      .add("Id",StringType(),True) \
      .add("fecNac",StringType(),True) \
      .add("sex",StringType(),True) \
      .add("ingresos",IntegerType(),True) \
      .add("partido",StringType(),True) \
      .add("edad",IntegerType(),True) \
      .add("mapa",StringType(),True)

"""
 * Bucle desde el año/mes 1921/01 a 2004/04
"""

primeraVez = True

for ano in range(1921, 2005):
    for mes in range(1, 13):
        if ano == 2004 and mes > 4: break
        
        if mes > 9: anomes = str(ano)+'-'+str(mes)
        else:       anomes = str(ano)+'-0'+str(mes)
    
        print(132*'='+'\nTratando partición yearMon='+anomes)
        inicioP = time.perf_counter()

        df = spark.read.option("mergeSchema", True).option("schema", "esquema").option("basePath", "file:///opt/workspace/datos/fecNac/fakePeople800M.parquet/")\
                       .format("parquet").load(['./datos/fecNac/fakePeople800M.parquet/yearMon='+anomes])

        print(df.count())
        
        # Join con el archivo del partido:
        df_sal = df.join(dfPart, df.partido == dfPart.partido, "inner").drop(dfPart.partido)
        df_sal.show(2, truncate=False)
        
        # Escritura del resultado por partición. En el primer mes borra todo lo anterior con overwrite:
        
        if primeraVez:
            df_sal.write.mode("overwrite").partitionBy("yearMon").parquet("./datos/fakePeopleConPartido.parquet")
            primeraVez = False
        else:
            df_sal.write.mode("append").partitionBy("yearMon").parquet("./datos/fakePeopleConPartido.parquet")

        print ('Final partición: '+time.strftime("%c"))
        final = time.perf_counter()
        print(f'Hecho en {round(final - inicioP, 4)} segundo(s)')


+-------+--------------------+--------------------+-------------+
|partido|              nombre|               lider|fechaCreacion|
+-------+--------------------+--------------------+-------------+
|    VOX|     VOX España S.A.|    Santiago Abascal|   2015-01-01|
|     PP|     Partido Popular|Alberto Núñez Feijóo|   1986-10-31|
|   PSOE|Partido Socialist...|       Pedro Sánchez|   1930-12-25|
|     CS|          Ciudadanos|      Inés Arrimadas|   2008-04-18|
|     UP|      Unidas Podemos|        Yolanda Díaz|   2014-04-15|
|  Otros|          Ni se sabe|               Naide|   1975-11-20|
+-------+--------------------+--------------------+-------------+

Tratando partición yearMon=1921-01


802938
+------------+----------+---+--------+-------+----+-------------+-------+---------------+----------------+-------------+
|Id          |fecNac    |sex|ingresos|partido|edad|mapa         |yearMon|nombre         |lider           |fechaCreacion|
+------------+----------+---+--------+-------+----+-------------+-------+---------------+----------------+-------------+
|ID1314544587|1921-01-03|H  |19100   |VOX    |101 |0-1-0-0-0-0-0|1921-01|VOX España S.A.|Santiago Abascal|2015-01-01   |
|ID1314546561|1921-01-21|H  |28000   |UP     |101 |1-1-1-0-0-0-0|1921-01|Unidas Podemos |Yolanda Díaz    |2014-04-15   |
+------------+----------+---+--------+-------+----+-------------+-------+---------------+----------------+-------------+
only showing top 2 rows



Final partición: Tue Nov  8 17:52:13 2022
Hecho en 8.7553 segundo(s)
Tratando partición yearMon=1921-02
802412
+------------+----------+---+--------+-------+----+-------------+-------+---------------+--------------------+-------------+
|Id          |fecNac    |sex|ingresos|partido|edad|mapa         |yearMon|nombre         |lider               |fechaCreacion|
+------------+----------+---+--------+-------+----+-------------+-------+---------------+--------------------+-------------+
|ID0075868439|1921-02-08|H  |13800   |PP     |101 |0-1-1-0-1-0-0|1921-02|Partido Popular|Alberto Núñez Feijóo|1986-10-31   |
|ID0078209400|1921-02-01|H  |20400   |VOX    |101 |1-1-1-0-1-0-0|1921-02|VOX España S.A.|Santiago Abascal    |2015-01-01   |
+------------+----------+---+--------+-------+----+-------------+-------+---------------+--------------------+-------------+
only showing top 2 rows



Final partición: Tue Nov  8 17:52:16 2022
Hecho en 2.6689 segundo(s)
Tratando partición yearMon=1921-03
804055
+------------+----------+---+--------+-------+----+-------------+-------+---------------------------------+--------------------+-------------+
|Id          |fecNac    |sex|ingresos|partido|edad|mapa         |yearMon|nombre                           |lider               |fechaCreacion|
+------------+----------+---+--------+-------+----+-------------+-------+---------------------------------+--------------------+-------------+
|ID1115985056|1921-03-30|H  |29500   |PP     |101 |0-0-0-0-1-0-0|1921-03|Partido Popular                  |Alberto Núñez Feijóo|1986-10-31   |
|ID1117557623|1921-03-18|M  |50000   |PSOE   |101 |0-0-0-0-1-0-1|1921-03|Partido Socialista Obrero Español|Pedro Sánchez       |1930-12-25   |
+------------+----------+---+--------+-------+----+-------------+-------+---------------------------------+--------------------+-------------+
only showing top 2 rows



Final partición: Tue Nov  8 17:52:18 2022
Hecho en 2.1763 segundo(s)
Tratando partición yearMon=1921-04
804336
+------------+----------+---+--------+-------+----+-------------+-------+---------------+----------------+-------------+
|Id          |fecNac    |sex|ingresos|partido|edad|mapa         |yearMon|nombre         |lider           |fechaCreacion|
+------------+----------+---+--------+-------+----+-------------+-------+---------------+----------------+-------------+
|ID1376029041|1921-04-02|M  |26100   |VOX    |101 |0-1-0-1-1-1-0|1921-04|VOX España S.A.|Santiago Abascal|2015-01-01   |
|ID1374500729|1921-04-17|H  |31100   |VOX    |101 |1-1-0-0-0-0-1|1921-04|VOX España S.A.|Santiago Abascal|2015-01-01   |
+------------+----------+---+--------+-------+----+-------------+-------+---------------+----------------+-------------+
only showing top 2 rows



...

Final partición: Tue Nov  8 22:06:04 2022
Hecho en 15.9161 segundo(s)
Tratando partición yearMon=2003-12


1068455
+------------+----------+---+--------+-------+----+-------------+-------+---------------+--------------------+-------------+
|Id          |fecNac    |sex|ingresos|partido|edad|mapa         |yearMon|nombre         |lider               |fechaCreacion|
+------------+----------+---+--------+-------+----+-------------+-------+---------------+--------------------+-------------+
|ID0022725777|2003-12-23|H  |14800   |PP     |18  |1-0-0-1-1-1-0|2003-12|Partido Popular|Alberto Núñez Feijóo|1986-10-31   |
|ID0025629294|2003-12-06|H  |18500   |VOX    |18  |1-1-0-0-0-0-0|2003-12|VOX España S.A.|Santiago Abascal    |2015-01-01   |
+------------+----------+---+--------+-------+----+-------------+-------+---------------+--------------------+-------------+
only showing top 2 rows



Final partición: Tue Nov  8 22:06:19 2022
Hecho en 14.4524 segundo(s)
Tratando partición yearMon=2004-01
267302
+------------+----------+---+--------+-------+----+-------------+-------+---------------+----------------+-------------+
|Id          |fecNac    |sex|ingresos|partido|edad|mapa         |yearMon|nombre         |lider           |fechaCreacion|
+------------+----------+---+--------+-------+----+-------------+-------+---------------+----------------+-------------+
|ID0025281031|2004-01-06|M  |18300   |UP     |18  |0-0-1-1-1-0-0|2004-01|Unidas Podemos |Yolanda Díaz    |2014-04-15   |
|ID0025281917|2004-01-23|H  |12300   |VOX    |18  |1-1-0-1-1-1-1|2004-01|VOX España S.A.|Santiago Abascal|2015-01-01   |
+------------+----------+---+--------+-------+----+-------------+-------+---------------+----------------+-------------+
only showing top 2 rows

Final partición: Tue Nov  8 22:06:20 2022
Hecho en 1.1297 segundo(s)
Tratando partición yearMon=2004-02
267214
+------------+----------+-

Final partición: Tue Nov  8 22:06:22 2022
Hecho en 1.4654 segundo(s)
Tratando partición yearMon=2004-03
266994
+------------+----------+---+--------+-------+----+-------------+-------+---------------------------------+-------------+-------------+
|Id          |fecNac    |sex|ingresos|partido|edad|mapa         |yearMon|nombre                           |lider        |fechaCreacion|
+------------+----------+---+--------+-------+----+-------------+-------+---------------------------------+-------------+-------------+
|ID0038919533|2004-03-31|M  |45200   |UP     |18  |1-1-1-1-0-1-1|2004-03|Unidas Podemos                   |Yolanda Díaz |2014-04-15   |
|ID0038920350|2004-03-06|H  |20400   |PSOE   |18  |1-1-0-0-0-0-1|2004-03|Partido Socialista Obrero Español|Pedro Sánchez|1930-12-25   |
+------------+----------+---+--------+-------+----+-------------+-------+---------------------------------+-------------+-------------+
only showing top 2 rows



Final partición: Tue Nov  8 22:06:23 2022
Hecho en 1.352 segundo(s)
Tratando partición yearMon=2004-04
266671
+------------+----------+---+--------+-------+----+-------------+-------+---------------------------------+--------------+-------------+
|Id          |fecNac    |sex|ingresos|partido|edad|mapa         |yearMon|nombre                           |lider         |fechaCreacion|
+------------+----------+---+--------+-------+----+-------------+-------+---------------------------------+--------------+-------------+
|ID0002126251|2004-04-21|M  |75900   |PSOE   |18  |0-0-0-0-1-0-1|2004-04|Partido Socialista Obrero Español|Pedro Sánchez |1930-12-25   |
|ID0001124858|2004-04-20|M  |20100   |CS     |18  |0-0-1-0-1-1-0|2004-04|Ciudadanos                       |Inés Arrimadas|2008-04-18   |
+------------+----------+---+--------+-------+----+-------------+-------+---------------------------------+--------------+-------------+
only showing top 2 rows

Final partición: Tue Nov  8 22:06:24 2022
H

In [4]:
print ('Final: '+time.strftime("%c"))
final = time.perf_counter()
print(f'Hecho en {round(final - inicio, 4)} segundo(s)')

Final: Tue Nov  8 22:06:24 2022
Hecho en 15263.1289 segundo(s)


In [5]:
spark.sparkContext.stop()
print('Sacabao')

Sacabao
